In [4]:
import os 
import numpy as np
from numpy import ndarray
from nsddatapaper_rsa.utils.nsd_get_data import get_conditions, get_labels
from nsddatapaper_rsa.utils.utils import category_dict, mds
from utils.utils import *
from nsd_access import NSDAccess 

In [5]:
subject = 1
roi = 'V0-1'
sub, n_sessions  = subjects_sessions[subject]

nsda = NSDAccess(data_dir)

https://github.com/charnley/rmsd/blob/master/rmsd/calculate_rmsd.py
https://stats.stackexchange.com/questions/186111/find-the-rotation-between-set-of-points
https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.transform.Rotation.align_vectors.html

In [34]:
def centroid(X: ndarray) -> ndarray:
    """
    Centroid is the mean position of all the points in all of the coordinate
    directions, from a vectorset X.

    https://en.wikipedia.org/wiki/Centroid

    C = sum(X)/len(X)

    Parameters
    ----------
    X : array
        (N,D) matrix, where N is points and D is dimension.

    Returns
    -------
    C : ndarray
        centroid
    """
    C: ndarray = X.mean(axis=0)
    return C

def kabsch2D(source: ndarray, target:ndarray) -> ndarray:
    """
    Find optimal rotation matrix for sourcd toward target
    Three steps: 
    - find the centroids of both arrays 
    - compute their covariance matrix
    - then find optimal rotation U 

    Parameters
    ----------
    source : array
        (N, D) matrix, N is points and D is dimension

    target : array
        (N, D) matrix, N is points and D is dimension

    Returns : 
    rotated_source: array,
        (N, D) matrix, rotated sources based on target, using kabsch algorithm
        see more: http://en.wikipedia.org/wiki/Kabsch_algorithm

    """
    source_C = source - centroid(source) 
    target_C = target - centroid(target)
    
    # This should be D*D and not N*N ! 
    cov = np.dot(source_C.T, target_C)

    # Compute the optimal rotation matrix U using singluar value decomposition (SVD) 
    # This might need a correction depending on the sign, check later if we run into issues
    U, S, V = np.linalg.svd(cov)

    U = np.dot(V, U.T)
    return U 
    

def rotate(source: ndarray, U: ndarray):
    """
    rotate the source matrix using the U optimal rotation matrix found
    using the kabasch2D function

    Parameters
    -----------
    source : array
        (N, D) matrix, N is points and D is dimension

    U : array
        (D, D) optimal rotation matrix

    Returns
    --------
    source_rotated : array
        (N, D) matrix, N is points and D is dimension
    """
    return np.dot(source, U)
    

In [22]:
mds_v1 = np.load('data/MDS/betas_split/subj01_37_V1_mds_betas_train.npy', allow_pickle=False)
mds_v1.shape

(9841, 2)

In [11]:
mds_v2 = np.load('data/MDS/betas_split/subj01_37_V2_mds_betas_train.npy', allow_pickle=False)
mds_v2.shape

(9841, 2)

In [29]:
U = kabsch2D(mds_v1, mds_v2)
U

array([[ 0.9937364 , -0.11175022],
       [-0.11175022, -0.9937364 ]], dtype=float32)

In [32]:
mds_v1_rotated = rotate(mds_v1, U)
mds_v1_rotated.shape

(9841, 2)